In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import seaborn as sns
import tqdm
import re

from matplotlib import pyplot as plt

%matplotlib inline

sns.set(style='whitegrid')
import matplotlib
matplotlib.rcParams["figure.dpi"] = 300
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'
plt.rcParams['savefig.facecolor'] = 'white'

In [2]:
import sys 
sys.path.append('../')

#### Imports

In [3]:
import os
import numpy as np
import torch
import torch.nn as nn
import wandb

import tqdm
import pickle
import pytorch_lightning as pl
import random

from torch.utils.data import IterableDataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from functools import partial
from collections import namedtuple
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [4]:
from src.transactions_qa.tqa_model import TransactionQAModel
from src.models.components.models import TransactionsModel
from src.utils.tools import (make_time_batch, 
                   calculate_embedding_size)

from src.data.alfa.components import ( 
                             cat_features_names, 
                             num_features_names, 
                             meta_features_names)

from src.data import AlfaDataModule 
from src.transactions_qa.tqa_model import TransactionQAModel
from src.transactions_qa.utils import get_projections_maps, get_exponent_number, get_mantissa_number
from src.tasks import AbstractTask, AutoTask
from src.transactions_qa.utils import get_split_indices,  prepare_splitted_batch, collate_batch_dict

In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoConfig

In [6]:
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [7]:
def load_transaction_model(encoder_type='whisper/tiny', head_type='next'):
    projections_maps = get_projections_maps(relative_folder="..")
    # Loading Transactions model & weights
    print(f"Loading Transactions model...")

    transactions_model_encoder_type = encoder_type
    transactions_model_head_type = head_type


    transactions_model_config = {
        "cat_features": cat_features_names,
        "cat_embedding_projections": projections_maps.get('cat_embedding_projections'),
        "num_features": num_features_names,
        "num_embedding_projections": projections_maps.get('num_embedding_projections'),
        "meta_features": meta_features_names,
        "meta_embedding_projections": projections_maps.get('meta_embedding_projections'),
        "encoder_type": transactions_model_encoder_type,
        "head_type": transactions_model_head_type,
        "embedding_dropout": 0.1
    }
    transactions_model = TransactionsModel(**transactions_model_config)

    return transactions_model, projections_maps

In [8]:
def load_language_model(language_model_name_or_path="google/flan-t5-small"):
    use_fast_tokenizer = True

    print(f"Loading Language model: `{language_model_name_or_path}`...")
    config_kwargs = {
        "use_auth_token": None,
        "return_unused_kwargs": True
    }

    tokenizer_kwargs = {
        "use_fast": use_fast_tokenizer,
        "use_auth_token": None,
        "do_lowercase": False
    }

    config, unused_kwargs = AutoConfig.from_pretrained(
        language_model_name_or_path, **config_kwargs
    )
    # Download vocabulary from huggingface.co and define model-specific arguments
    tokenizer = AutoTokenizer.from_pretrained(language_model_name_or_path, **tokenizer_kwargs)

    # Download model from huggingface.co and cache.
    lm_model = AutoModelForSeq2SeqLM.from_pretrained(
        language_model_name_or_path,
        config=config
    )
    return lm_model, tokenizer

In [9]:
def load_datamodule():
    DATA_PATH = '/home/jovyan/romashka/data' 
    dataset_config = {
                'data_dir': DATA_PATH,
                'batch_size': 32,
                'min_seq_len': 0,
                'max_seq_len': 250,
                'shuffle': True,
                'num_workers': 5,
                'pin_memory': True,
                'seed': 42
    }    

    dm = AlfaDataModule(**dataset_config)
    return dm

In [10]:
def load_tasks(task_names, tokenizer):
    # Create tasks
    tasks = []
    tasks_kwargs = [{"num_options": 6, "floating_threshold": True, 'answer2text': False, 'use_numerical_output': False}, 
    {"num_options": 6, "floating_threshold": False, 'use_numerical_output': False}] # ground truth + 5 additional options
    if isinstance(task_names, str):
        task_names = eval(task_names)
    if isinstance(tasks_kwargs, str):
        tasks_kwargs = eval(tasks_kwargs)
    print(f"Got task_names: {task_names} with task_kwargs: {tasks_kwargs}")

    for task_i, task_name in enumerate(task_names):
        task_kwargs = tasks_kwargs[task_i] if task_i < len(tasks_kwargs) else {}
        if "tokenizer" not in task_kwargs:
            task_kwargs['tokenizer'] = tokenizer
        task = AutoTask.get(task_name=task_name, **task_kwargs)
        tasks.append(task)
    print(f"Created {len(tasks)} tasks.")
    return tasks

In [11]:
def make_tqa_model(lm_model, transactions_model, tokenizer, tasks):
    # Create general Tranactions QA model
    max_steps = 100_000
    warmup_steps = 1000
    do_freeze_transactions_model = True
    do_freeze_language_model = True
    do_freeze_connector = False

    transactionsQA_model_config = {
        "warmup_steps": warmup_steps,
        "training_steps": max_steps,
        "do_freeze_tm": do_freeze_transactions_model,
        "do_freeze_lm": do_freeze_language_model,
        "do_freeze_connector": do_freeze_connector,
        "connector_input_size": 384,
        "use_numerical_input": False,
        "use_numerical_output": False,
        "numerical_context": "context",
    }

    model = TransactionQAModel(
        language_model=lm_model,
        transaction_model=transactions_model,
        tokenizer=tokenizer,
        tasks=tasks,
        **transactionsQA_model_config
    )
    return model

In [41]:
# task_names = ['next_transactions_30_days_binary', 'default', 'next_mcc_binary', 'next_mcc_open_ended']
open_ended_task_names = ['next_amnt_open_ended']
binary_task_names = ['next_amnt_binary']

LM_NAME = 'google/flan-t5-small'

lm_model, tokenizer = load_language_model(language_model_name_or_path=LM_NAME)
transactions_model, projections_maps = load_transaction_model()

open_end_tasks = load_tasks(open_ended_task_names, tokenizer)
binary_tasks = load_tasks(binary_task_names, tokenizer)
dm = load_datamodule()

open_ended_model = make_tqa_model(lm_model, transactions_model, tokenizer, open_end_tasks)
binary_model = make_tqa_model(lm_model, transactions_model, tokenizer, binary_tasks)

Loading Language model: `google/flan-t5-small`...
Loading Transactions model...


2023-06-25 23:38:13,980 - [INFO] - Tasks - (task_abstract.py).generate_question_templates(206) - Given 5 starting options and 1 ending options results in 5 total combinations.
2023-06-25 23:38:13,982 - [INFO] - Tasks - (task_abstract.py).extend_vocabulary(173) - Added to tokenizer: 2 tokens.
2023-06-25 23:38:13,983 - [INFO] - Tasks - (task_abstract.py).extend_vocabulary(179) - Notice: resize_token_embeddings of a model to adapt to the size of the new vocabulary!
2023-06-25 23:38:13,986 - [INFO] - Tasks - (task_abstract.py).generate_question_templates(206) - Given 5 starting options and 1 ending options results in 5 total combinations.
2023-06-25 23:38:13,987 - [INFO] - Tasks - (task_abstract.py).extend_vocabulary(173) - Added to tokenizer: 0 tokens.
2023-06-25 23:38:13,988 - [INFO] - Tasks - (task_abstract.py).extend_vocabulary(179) - Notice: resize_token_embeddings of a model to adapt to the size of the new vocabulary!
2023-06-25 23:38:13,989 - [INFO] - Tasks - (task_abstract.py).gene

USING whisper
Got task_names: ['next_amnt_open_ended'] with task_kwargs: [{'num_options': 6, 'floating_threshold': True, 'answer2text': False, 'use_numerical_output': False}, {'num_options': 6, 'floating_threshold': False, 'use_numerical_output': False}]
Created 1 tasks.
Got task_names: ['next_amnt_binary'] with task_kwargs: [{'num_options': 6, 'floating_threshold': True, 'answer2text': False, 'use_numerical_output': False}, {'num_options': 6, 'floating_threshold': False, 'use_numerical_output': False}]
Created 1 tasks.
Output dimension of embedding model: 384
Input dimension of autoregressive model: 512
Creating linear connector from 384 to 512 and move to device: cpu.
ModuleDict(
  (next_amnt_open_ended): ModuleDict(
    (mae): MeanAbsoluteError()
    (mape): MeanAbsolutePercentageError()
    (rouge): ROUGEScore()
  )
)


2023-06-25 23:38:14,252 - [INFO] - TransactionQAModel - (tqa_model.py)._resize_text_embeddings(263) - LM resized `num_embeddings`: 32102, `embedding_dim`: 512
2023-06-25 23:38:14,252 - [INFO] - TransactionQAModel - (tqa_model.py)._resize_text_embeddings(263) - LM resized `num_embeddings`: 32102, `embedding_dim`: 512
2023-06-25 23:38:14,252 - [INFO] - TransactionQAModel - (tqa_model.py)._resize_text_embeddings(263) - LM resized `num_embeddings`: 32102, `embedding_dim`: 512
2023-06-25 23:38:14,257 - [INFO] - TransactionQAModel - (tqa_model.py)._prepare_model(217) - Freezing transaction model's parameters...
2023-06-25 23:38:14,257 - [INFO] - TransactionQAModel - (tqa_model.py)._prepare_model(217) - Freezing transaction model's parameters...
2023-06-25 23:38:14,257 - [INFO] - TransactionQAModel - (tqa_model.py)._prepare_model(217) - Freezing transaction model's parameters...
2023-06-25 23:38:14,261 - [INFO] - TransactionQAModel - (tqa_model.py)._prepare_model(222) - Freezing language mode

Output dimension of embedding model: 384
Input dimension of autoregressive model: 512
Creating linear connector from 384 to 512 and move to device: cpu.
ModuleDict(
  (next_amnt_binary): ModuleDict(
    (auc): BinaryAUROC()
    (accuracy): BinaryAccuracy()
  )
)


In [48]:
open_ended_ckpt = torch.load('/home/jovyan/romashka/checkpoints/checkpoints/tqa_200k-steps_ft=all_numerical_v59/last.ckpt')['state_dict']
binary_ckpt = torch.load('/home/jovyan/romashka/checkpoints/checkpoints/tqa_200k-steps_ft=all_numerical_v58/last.ckpt')['state_dict']

open_ended_model.load_state_dict(open_ended_ckpt)
# binary_model.load_state_dict(binary_ckpt)

# open_ended_model.cuda();
# binary_model.cuda();

<All keys matched successfully>

### Numerical Input

In [43]:
batch = next(iter(dm.val_dataloader()))

In [44]:
import os
import contextlib
from contextlib import contextmanager

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [45]:
# t = tqa_model1.model_step(batch)

In [46]:
device = 'cuda:0'

In [47]:
@contextmanager
def nullcontext(enter_result=None):
    yield enter_result


In [38]:
open_ended_results = []
binary_results = []

with torch.no_grad():
    for batch in tqdm.tqdm(dm.val_dataloader()):
        open_ended_model.eval()
        binary_model.eval()

        for elem in batch:
            batch[elem] = batch[elem].to(device)
            

        
        # Removing prints
        # with open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
        with nullcontext():
            out1, batch_answers1 = open_ended_model.model_step(batch)
            pred1, true_output1 = open_ended_model.tasks[0].process_outputs(out1, batch_answers1)

            print(pred1)

            out2, batch_answers2 = binary_model.model_step(batch)
            pred2, true_output2 = binary_model.tasks[0].process_outputs(out2, batch_answers2)

            questions = binary_model.tokenizer.batch_decode(out2['question_encoded'], skip_special_tokens=True)
            thresholds = torch.tensor([float(re.findall("\d+\.\d+",  string)[0]) for string in questions], device=device)

            open_ended_predictions = (pred1 > thresholds) == true_output2
            binary_predictions = (pred2 > 0.5) == true_output2

            open_ended_results.append(open_ended_predictions)
            binary_results.append(binary_predictions)

1it [00:01,  1.21s/it]

Value is not a number: amount25
Value is not a number: amount25
tensor([125,  25,  25,  25,  25,  25,   0,  25,  25,  25,  25,  25,  25,  25,
        125,  25,   0,  25,   0,  25,  25,  25,   1,  25,  25,  25, 125,  25,
         25,   0,  25,  25], device='cuda:0')
tensor([ 25,  25, 125, 125, 125, 125,  25,  25, 125,  25,  25,  25,  25,  10,
        125, 125, 125, 125,   0,  25,  25, 125,  25, 125,  25, 125, 125, 125,
        125, 125, 125, 125], device='cuda:0')
tensor([ 25,  25,  25, 125,  25, 125, 125,  25, 125, 125, 125, 125,  10, 125,
         25, 125,  25,  10, 125,  25,  25, 125,  25,  25, 125, 125,  25,  25,
         25,  10,  25,  25], device='cuda:0')


5it [00:01,  4.73it/s]

tensor([125,  25,  25, 125, 125, 125,  25, 125,  25, 125, 125,  25,  25, 125,
         25, 125, 125,  25, 125, 125, 125, 125,  25, 125, 125,   0, 125, 125,
        125, 125, 125, 125], device='cuda:0')
tensor([125, 125, 125, 125, 125,  25, 125, 125,  25, 125,  25, 125,  25,  25,
        125,  10, 125,  25,  25, 125,  25, 125, 125,  25, 125,  25,  25,  25,
        125, 125, 125,  25], device='cuda:0')
tensor([ 10,  25,  25,  25,  10,  25, 125, 125, 125, 125,   0, 125, 125,  10,
         25,  10,  25, 125, 125, 125, 125,  25,  10, 125, 125, 125, 125,  25,
         25,  25,  25,  25], device='cuda:0')


9it [00:01,  8.39it/s]

...

231it [00:16, 14.85it/s]

Value is not a number: amount25
tensor([25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,  0, 25, 25, 25, 25, 25,
        25, 25, 25, 25, 25,  0,  1, 25, 25, 25,  0, 25, 25, 25],
       device='cuda:0')
Value is not a number: £0
Value is not a number: amount25
Value is not a number: amount25
Value is not a number: amount0
tensor([25, 25, 25, 25, 25, 25, 25,  0, 25, 25, 25, 25,  0, 25, 25, 25, 25, 25,
        25, 25,  0, 25, 25, 25, 25, 25, 25,  0, 25, 25, 25, 25],
       device='cuda:0')
Value is not a number: amount01
Value is not a number: amount25
Value is not a number: amount25
tensor([ 25,   0, 125,  25,  25,  25,  25,   0,  25,   1,  25,  25,  25,  25,
         25,  25,  25,   0,   0, 125, 101,  25,  25,  25,  25,  25,  25,  25,
         25,  25,  25,  25], device='cuda:0')
Value is not a number: amount0
Value is not a number: amount25
Value is not a number: amount25


233it [00:16, 13.73it/s]


tensor([25, 25, 25, 25, 25,  0, 25, 25, 25, 25,  0,  0, 25, 25, 25, 25, 25, 25,
        25, 25,  1, 10,  0, 25,  0, 25, 25, 25, 25, 25, 25,  0],
       device='cuda:0')
Value is not a number: amount25


KeyboardInterrupt: 

In [25]:
batch.keys()

dict_keys(['num_features', 'cat_features', 'meta_features', 'mask', 'app_id', 'label'])

In [20]:
torch.cat(binary_results).float().mean()

tensor(0.8757, device='cuda:0')

In [21]:
torch.cat(open_ended_results).float().mean()

tensor(0.4865, device='cuda:0')

### New metric

In [79]:
buckets = torch.linspace(0, 1, 10, device=device)

In [83]:
true_buckets = torch.bucketize(true_output, buckets)
predicted_buckets = torch.bucketize(pred1, buckets)

In [91]:
binary_accuracy = (10 - abs(true_buckets - predicted_buckets)).float().mean() / 10

In [134]:
for i in range(1, 10):
    thrs = torch.tensor([i / 10] * batch_size, device=device).unsqueeze(1)
    out2, batch_answers2 = binary_model.model_step(batch, threshold=thrs)
    pred2, true_output2 = binary_model.tasks[0].process_outputs(out2, batch_answers2)

    binary_predictions = (pred2 > 0.5) == true_output2

In [136]:
binary_predictions

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True], device='cuda:0')